In [ ]:
import requests
import itchat
from itchat.content import *
# Import necessary modules
from rasa_nlu.training_data import load_data
from rasa_nlu.config import RasaNLUModelConfig
from rasa_nlu.model import Trainer
from rasa_nlu import config

# Create a trainer that uses this config
trainer = Trainer(config.load("D:\\出国\\项目2\\code\\rasanlu\\rasa_nlu_chi\\sample_configs\\config_jieba_mitie_sklearn.yml"))

# Load the training data
training_data = load_data('D:\\出国\\项目2\\code\\rasanlu\\rasa_nlu_chi\\data\\examples\\rasa\\stock-rasa_ch.json')

# Create an interpreter by training the model
interpreter = trainer.train(training_data)

def en_ch_d(english_name):
    transplate_dict={'volume':'成交量',
                     'market_cap':'市值',
                     'high':'最高股价',
                     'last_trade_price':'当前股价'
                    }
    return transplate_dict[english_name]

def en_ch_c(english_name):
    transplate_dict={'F':'福特',
                     'FB':'脸书',
                     'AAPL':'苹果',
                     'MSFT':'微软'
                    }
    return transplate_dict[english_name]

@itchat.msg_register(TEXT)
def respond(msg):
    global response_dict
    global company_name
    times_comp=0
    times_det=0
    message=msg['Text']
    entities = interpreter.parse(message)["entities"]
    for ents in entities:
        if ents['entity']=='company':
            comp=ents['value']
            url='https://api.robinhood.com/fundamentals/{}/'.format(comp)
            r=requests.get(url)
            response_dict1=r.json()
            url='https://api.robinhood.com/quotes/{}/'.format(comp)
            r=requests.get(url)
            response_dict2=r.json()
            response_dict=dict(response_dict1,**response_dict2)
            company_name=en_ch_c(comp)
            times_comp=1
            for ents in entities:
                if ents['entity']=='detail':
                    detail=ents['value']
                    detail_name=en_ch_d(detail)
                    response="{0}的{1}是:{2}".format(company_name, detail_name, response_dict[detail])
                    times_det=1
                    itchat.send(response, toUserName=msg['FromUserName'])
            if times_det==0:
                response="您想知道关于这个公司的什么信息?"
                itchat.send(response, toUserName=msg['FromUserName'])
    
    if times_comp==0:
        for ents in entities:
            detail=ents['value']
            detail_name=en_ch_d(detail)
            response="{0}的{1}是:{2}".format(company_name, detail_name, response_dict[detail])
            itchat.send(response, toUserName=msg['FromUserName'])

itchat.auto_login()
itchat.run()
